This notebook is to scrape football data from fbref.com

Much of the scraping code is taken from this repository: https://github.com/chmartin/FBref_EPL.

Run the first cell and then the further cells to get data for whichever leagues you want.

All data is courtesy of StatsBomb via FBref. Find me on Twitter @pathaleee!


In [ ]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
ERROR: Operation cancelled by user
^C
Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libgtk-4.so.1                                    ║
║     libgraphene-1.0.so.0                             ║
║     libwoff2dec.so.1.0.2                             ║
║     libvpx.so.9                                      ║
║     libevent-2.1.so.7                                ║
║     libgstallocators-1.0.so.0                        ║
║     libgstapp-1.0.so.0                               ║
║     libgstpbutils-1.0.so.0                           ║
║     libgstaudio-1.0.so.0                             ║
║     libgsttag-1.0.so.0                               ║
║     libgstvideo-1.0.so.0                             ║
║     libgstgl-1.0.so.0                          

In [72]:
#standard(stats)
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]
stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"] 
#goalkeeping(keepers)
keepers = ["player","nationality","position","squad","age","birth_year","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
keepers3 = ["players_used","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
#advance goalkeeping(keepersadv)
keepersadv = ["player","nationality","position","squad","age","birth_year","minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]
keepersadv2 = ["minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]
#shooting(shooting)
shooting = ["player","nationality","position","squad","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
#passing(passing)
passing = ["player","nationality","position","squad","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
#passtypes(passing_types)
passing_types = ["player","nationality","position","squad","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
#goal and shot creation(gca)
gca = ["player","nationality","position","squad","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
#defensive actions(defense)
defense = ["player","nationality","position","squad","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
#possession(possession)
possession = ["player","nationality","position","squad","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
#playingtime(playingtime)
playingtime = ["player","nationality","position","squad","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
#miscallaneous(misc)
misc = ["player","nationality","position","squad","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time

# --- Selenium Setup ---
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) "
                     "Chrome/124.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)


# --- Core function using Selenium instead of requests ---
def get_tables(url, text):
    driver.get(url)
    time.sleep(3)  # give JS time to load
    html = driver.page_source
    print(f'getting table from : {url}')
    soup = BeautifulSoup(html, "lxml")
    all_tables = soup.find_all("tbody")

    if len(all_tables) < 3:
        raise ValueError("Could not find expected tables")

    team_table = all_tables[0]
    team_vs_table = all_tables[1]
    player_table = all_tables[2]

    if text == "for":
        return player_table, team_table
    if text == "vs":
        return player_table, team_vs_table
    
    driver.quit()


# --- Keep your helper functions (unchanged) ---
def get_frame(endpoint,features, player_table):
    print(f'getting {features} from {endpoint}')
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all("tr")
    for row in rows_player:
        if row.find("th", {"scope": "row"}) is not None:
            for f in features_wanted_player:
                cell = row.find("td", {"data-stat": f})
                text = cell.text.strip() if cell else "0"
                if text == "":
                    text = "0"
                if f not in ["player","nationality","position","squad","age","birth_year"]:
                    text = float(text.replace(",", "")) if text.replace(".", "", 1).isdigit() else text
                pre_df_player.setdefault(f, []).append(text)
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player


def get_frame_team(endpoint,features, team_table):
    print(f'getting {features} from {endpoint}')
    pre_df_squad = dict()
    rows_squad = team_table.find_all("tr")
    for row in rows_squad:
        if row.find("th", {"scope": "row"}) is not None:
            name = row.find("th", {"data-stat": "team"}).text.strip()
            pre_df_squad.setdefault("team", []).append(name)
            for f in features:
                cell = row.find("td", {"data-stat": f})
                text = cell.text.strip() if cell else "0"
                if text == "":
                    text = "0"
                if f not in ["player","nationality","position","squad","age","birth_year"]:
                    text = float(text.replace(",", "")) if text.replace(".", "", 1).isdigit() else text
                pre_df_squad.setdefault(f, []).append(text)
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad


# --- Category helper wrappers ---
def frame_for_category(category, top, end, features):
    url = f"{top}{category}{end}"
    player_table, team_table = get_tables(url, "for")
    return get_frame(end,features, player_table)


def frame_for_category_team(category, top, end, features, text):
    url = f"{top}{category}{end}"
    player_table, team_table = get_tables(url, text)
    return get_frame_team(end,features, team_table)



def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end,stats)
    df2 = frame_for_category('shooting',top,end,shooting2)
    df3 = frame_for_category('passing',top,end,passing2)
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    df5 = frame_for_category('gca',top,end,gca2)
    df6 = frame_for_category('defense',top,end,defense2)
    df7 = frame_for_category('possession',top,end,possession2)
    df8 = frame_for_category('misc',top,end,misc2)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top,end):
    df1 = frame_for_category('keepers',top,end,keepers)
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df



#Function to get team-wise data accross all categories as mentioned above
def get_team_data(top,end,text):
    df1 = frame_for_category_team('stats',top,end,stats3,text)
    df2 = frame_for_category_team('keepers',top,end,keepers3,text)
    df3 = frame_for_category_team('keepersadv',top,end,keepersadv2,text)
    df4 = frame_for_category_team('shooting',top,end,shooting3,text)
    df5 = frame_for_category_team('passing',top,end,passing2,text)
    df6 = frame_for_category_team('passing_types',top,end,passing_types2,text)
    df7 = frame_for_category_team('gca',top,end,gca2,text)
    df8 = frame_for_category_team('defense',top,end,defense2,text)
    df9 = frame_for_category_team('possession',top,end,possession2,text)
    df10 = frame_for_category_team('misc',top,end,misc2,text)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


In [ ]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')

#Save csv file to Desktop
df_outfield.to_csv('PL2021_Outfield.csv',index=False)

KeyboardInterrupt: 

In [55]:
#df_outfield.head()
pd.set_option("display.max_columns", None)
df_outfield[df_outfield["player"] == "Erling Haaland"]


,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,assists,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,minutes_90s,shots_total,shots_on_target,shots_free_kicks,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,npxg_per_shot,xg_net,npxg_net,passes_completed,passes,passes_pct,passes_total_distance,passes_progressive_distance,passes_completed_short,passes_short,passes_pct_short,passes_completed_medium,passes_medium,passes_pct_medium,passes_completed_long,passes_long,passes_pct_long,xa_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes,passes_live,passes_dead,passes_free_kicks,through_balls,passes_pressure,passes_switches,crosses,corner_kicks,corner_kicks_in,corner_kicks_out,corner_kicks_straight,passes_ground,passes_low,passes_high,passes_left_foot,passes_right_foot,passes_head,throw_ins,passes_other_body,passes_offsides,passes_oob,passes_intercepted,passes_blocked,sca,sca_per90,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,gca,gca_per90,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_defense,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,dribble_tackles,dribbles_vs,dribble_tackles_pct,dribbled_past,pressures,pressure_regains,pressure_regain_pct,pressures_def_3rd,pressures_mid_3rd,pressures_att_3rd,blocks,blocked_shots,blocked_shots_saves,blocked_passes,interceptions,clearances,errors,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,dribbles_completed_pct,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,pass_targets,passes_received,passes_received_pct,miscontrols,dispossessed,cards_yellow_red,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
162,Erling Haaland,no NOR,FW,0,25-105,2000,10.0,10.0,854.0,13.0,1.0,0.0,0.0,0.0,0.0,1.37,0.11,1.48,1.37,1.48,10.2,10.2,0.0,1.08,0.0,0.0,1.08,0.0,9.5,0.0,25.0,0.0,58.1,0.0,2.63,0.3,0.52,0.24,+2.8,+2.8,77.0,115.0,67.0,804.0,196.0,49.0,71.0,69.0,15.0,24.0,62.5,0.0,2.0,0.0,0.0,4.0,6.0,1.0,0.0,6.0,105.0,10.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,16.0,1.69,9.0,0.0,0.0,4.0,0.0,3.0,0.32,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,19.0,0.0,215.0,21.0,32.0,82.0,103.0,66.0,215.0,0.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,22.0,7.0,14.0,0.0,153.0,0.0,13.0,5.0,0.0,7.0,6.0,0.0,0.0,0.0,0.0,14.0,24.0,9.0,72.7


In [65]:
#This cell is to get the outfield player data for any competition

#Go to the 'Standard stats' page of the league
#For Premier League 2020/21, the link is this: https://fbref.com/en/comps/9/stats/Premier-League-Stats
#Remove the 'stats', and pass the first and third part of the link as parameters like below


PL_outfield_25_26 = get_outfield_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')
PL_outfield_24_25 = get_outfield_data('https://fbref.com/en/comps/9/2024-2025/','/2024-2025-Premier-League-Stats')
PL_outfield_23_24 = get_outfield_data('https://fbref.com/en/comps/9/2023-2024/','/2023-2024-Premier-League-Stats')
PL_outfield_22_23 = get_outfield_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats')
PL_outfield_21_22 = get_outfield_data('https://fbref.com/en/comps/9/2021-2022/','/2021-2022-Premier-League-Stats')
PL_outfield_20_21 = get_outfield_data('https://fbref.com/en/comps/9/2020-2021/','/2020-2021-Premier-League-Stats')

#Save csv file to Desktop
PL_outfield_25_26.to_csv('PL_outfield_25_26.csv',index=False)
PL_outfield_24_25.to_csv('PL_outfield_24_25.csv',index=False)
PL_outfield_23_24.to_csv('PL_outfield_23_24.csv',index=False)
PL_outfield_22_23.to_csv('PL_outfield_22_23.csv',index=False)
PL_outfield_21_22.to_csv('PL_outfield_21_22.csv',index=False)
PL_outfield_20_21.to_csv('PL_outfield_20_21.csv',index=False)



getting table from : https://fbref.com/en/comps/9/stats/Premier-League-Stats
getting ['player', 'nationality', 'position', 'squad', 'age', 'birth_year', 'games', 'games_starts', 'minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'cards_yellow', 'cards_red', 'goals_per90', 'assists_per90', 'goals_assists_per90', 'goals_pens_per90', 'goals_assists_pens_per90', 'xg', 'npxg', 'xa', 'xg_per90', 'xa_per90', 'xg_xa_per90', 'npxg_per90', 'npxg_xa_per90'] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/shooting/Premier-League-Stats
getting ['minutes_90s', 'goals', 'pens_made', 'pens_att', 'shots_total', 'shots_on_target', 'shots_free_kicks', 'shots_on_target_pct', 'shots_total_per90', 'shots_on_target_per90', 'goals_per_shot', 'goals_per_shot_on_target', 'xg', 'npxg', 'npxg_per_shot', 'xg_net', 'npxg_net'] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/passing/Premier-League-Stats
getting ['passes_completed', 'passes', 'passes_pct'

In [69]:
#This cell is to get the goalkeeper data for any competition

#Go to the 'Standard stats' page of the league
#For Premier League 2020/21, the link is this: https://fbref.com/en/comps/9/stats/Premier-League-Stats
#Remove the 'stats', and pass the first and third part of the link as parameters like below
PL_keeper_25_26 = get_keeper_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')
PL_keeper_24_25 = get_keeper_data('https://fbref.com/en/comps/9/2024-2025/','/2024-2025-Premier-League-Stats')
PL_keeper_23_24 = get_keeper_data('https://fbref.com/en/comps/9/2023-2024/','/2023-2024-Premier-League-Stats')
PL_keeper_22_23 = get_keeper_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats')
PL_keeper_21_22 = get_keeper_data('https://fbref.com/en/comps/9/2021-2022/','/2021-2022-Premier-League-Stats')
PL_keeper_20_21 = get_keeper_data('https://fbref.com/en/comps/9/2020-2021/','/2020-2021-Premier-League-Stats')

#Save csv file to Desktop
PL_keeper_25_26.to_csv('PL_keeper_25_26.csv',index=False)
PL_keeper_24_25.to_csv('PL_keeper_24_25.csv',index=False)
PL_keeper_23_24.to_csv('PL_keeper_23_24.csv',index=False)
PL_keeper_22_23.to_csv('PL_keeper_22_23.csv',index=False)
PL_keeper_21_22.to_csv('PL_keeper_21_22.csv',index=False)
PL_keeper_20_21.to_csv('PL_keeper_20_21.csv',index=False)

getting table from : https://fbref.com/en/comps/9/keepers/Premier-League-Stats
getting ['player', 'nationality', 'position', 'squad', 'age', 'birth_year', 'games_gk', 'games_starts_gk', 'minutes_gk', 'goals_against_gk', 'goals_against_per90_gk', 'shots_on_target_against', 'saves', 'save_pct', 'wins_gk', 'draws_gk', 'losses_gk', 'clean_sheets', 'clean_sheets_pct', 'pens_att_gk', 'pens_allowed', 'pens_saved', 'pens_missed_gk'] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/keepersadv/Premier-League-Stats
getting ['minutes_90s', 'goals_against_gk', 'pens_allowed', 'free_kick_goals_against_gk', 'corner_kick_goals_against_gk', 'own_goals_against_gk', 'psxg_gk', 'psnpxg_per_shot_on_target_against', 'psxg_net_gk', 'psxg_net_per90_gk', 'passes_completed_launched_gk', 'passes_launched_gk', 'passes_pct_launched_gk', 'passes_gk', 'passes_throws_gk', 'pct_passes_launched_gk', 'passes_length_avg_gk', 'goal_kicks', 'pct_goal_kicks_launched', 'goal_kick_length_avg', 'cro

KeyboardInterrupt: 

In [73]:
#This cell is to get the data FOR all teams in any competition

#Go to the 'Standard stats' page of the league
#For Premier League 2020/21, the link is this: https://fbref.com/en/comps/9/stats/Premier-League-Stats
#Remove the 'stats', and pass the first and third part of the link as parameters like below
#df_team = get_team_data('https://fbref.com/en/comps/9/','/Premier-League-Stats','for')

PL_team_25_26 = get_team_data('https://fbref.com/en/comps/9/','/Premier-League-Stats','for')
PL_team_24_25 = get_team_data('https://fbref.com/en/comps/9/2024-2025/','/2024-2025-Premier-League-Stats','for')
PL_team_23_24 = get_team_data('https://fbref.com/en/comps/9/2023-2024/','/2023-2024-Premier-League-Stats','for')
PL_team_22_23 = get_team_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats','for')
PL_team_21_22 = get_team_data('https://fbref.com/en/comps/9/2021-2022/','/2021-2022-Premier-League-Stats','for')
PL_team_20_21 = get_team_data('https://fbref.com/en/comps/9/2020-2021/','/2020-2021-Premier-League-Stats','for')

#Save csv file to Desktop
PL_team_25_26.to_csv('PL_team_25_26.csv',index=False)
PL_team_24_25.to_csv('PL_team_24_25.csv',index=False)
PL_team_23_24.to_csv('PL_team_23_24.csv',index=False)
PL_team_22_23.to_csv('PL_team_22_23.csv',index=False)
PL_team_21_22.to_csv('PL_team_21_22.csv',index=False)
PL_team_20_21.to_csv('PL_team_20_21.csv',index=False)

getting table from : https://fbref.com/en/comps/9/stats/Premier-League-Stats
getting ['players_used', 'possession', 'games', 'games_starts', 'minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'cards_yellow', 'cards_red', 'goals_per90', 'assists_per90', 'goals_assists_per90', 'goals_pens_per90', 'goals_assists_pens_per90', 'xg', 'npxg', 'xa', 'xg_per90', 'xa_per90', 'xg_xa_per90', 'npxg_per90', 'npxg_xa_per90'] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/keepers/Premier-League-Stats
getting ['players_used', 'games_gk', 'games_starts_gk', 'minutes_gk', 'goals_against_gk', 'goals_against_per90_gk', 'shots_on_target_against', 'saves', 'save_pct', 'wins_gk', 'draws_gk', 'losses_gk', 'clean_sheets', 'clean_sheets_pct', 'pens_att_gk', 'pens_allowed', 'pens_saved', 'pens_missed_gk'] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/keepersadv/Premier-League-Stats
getting ['minutes_90s', 'goals_against_gk', 'pens_allowed', 'free_k